In [4]:
from google.colab import drive
import shutil
import os
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import ModelCheckpoint
from pathlib import Path

In [5]:

# Set paths for convenience
zip_path = "/content/drive/MyDrive/cifake.zip"
unzip_path = "/content/drive/MyDrive/cifake_dataset"
# Define paths for train, validation, and test sets
train_path = Path(unzip_path) / 'train'
validation_path = Path(unzip_path) / 'validation'
test_path = Path(unzip_path) / 'test'

In [6]:
# Mount Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
# Remove the unzip path if it already exists to avoid conflicts
if os.path.exists(unzip_path):
    shutil.rmtree(unzip_path)

# Unzip the dataset using shutil
shutil.unpack_archive(zip_path, unzip_path, 'zip')

print("Dataset unzipped at:", unzip_path)


Dataset unzipped at: /content/drive/MyDrive/cifake_dataset


In [8]:
# Create validation folders if they don't exist
(validation_path / 'REAL').mkdir(parents=True, exist_ok=True)
(validation_path / 'FAKE').mkdir(parents=True, exist_ok=True)

# Move 10,000 images from train/REAL to validation/REAL
real_images = list((train_path / 'REAL').glob('*.jpg'))
for img_path in real_images[:10000]:
    shutil.move(str(img_path), validation_path / 'REAL')

# Move 10,000 images from train/FAKE to validation/FAKE
fake_images = list((train_path / 'FAKE').glob('*.jpg'))
for img_path in fake_images[:10000]:
    shutil.move(str(img_path), validation_path / 'FAKE')

print("Images organized into training, validation, and test folders.")


Images organized into training, validation, and test folders.


In [9]:
# Load datasets
train_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    train_path,
    image_size=(32, 32),
    batch_size=32,
    label_mode='binary'
)

Found 80000 files belonging to 2 classes.


In [10]:
validation_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    validation_path,
    image_size=(32, 32),
    batch_size=32,
    label_mode='binary'
)

Found 20000 files belonging to 2 classes.


In [11]:
test_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    test_path,
    image_size=(32, 32),
    batch_size=32,
    label_mode='binary'
)

Found 20000 files belonging to 2 classes.


In [12]:

# Define the model architecture
model = models.Sequential([
    layers.Rescaling(1./255, input_shape=(32, 32, 3)),  # Rescale
    layers.Conv2D(32, (3, 3), activation='relu'),       # Conv2D
    layers.MaxPooling2D((2, 2)),                        # Max Pool
    layers.Conv2D(32, (3, 3), activation='relu'),       # Conv2D
    layers.MaxPooling2D((2, 2)),                        # Max Pool
    layers.Flatten(),                                   # Flatten
    layers.Dense(64, activation='relu'),                # Dense
    layers.Dense(1, activation='sigmoid')               # Output layer with Sigmoid
])

/usr/local/lib/python3.10/dist-packages/keras/src/layers/preprocessing/tf_data_layer.py:19: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [13]:

# Compile the model
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [16]:
# Set up a callback to save the best model
checkpoint = ModelCheckpoint('/content/drive/MyDrive/cifake_best_model.keras',
                             monitor='val_loss',
                             save_best_only=True,
                             mode='min')


In [17]:
# Train the model
history = model.fit(
    train_dataset,
    validation_data=validation_dataset,
    epochs=30,
    callbacks=[checkpoint]
)

Epoch 1/30
2500/2500 ━━━━━━━━━━━━━━━━━━━━ 146s 57ms/step - accuracy: 0.7983 - loss: 0.4250 - val_accuracy: 0.8982 - val_loss: 0.2443
Epoch 2/30
2500/2500 ━━━━━━━━━━━━━━━━━━━━ 164s 66ms/step - accuracy: 0.8953 - loss: 0.2526 - val_accuracy: 0.9179 - val_loss: 0.2066
Epoch 3/30
2500/2500 ━━━━━━━━━━━━━━━━━━━━ 161s 64ms/step - accuracy: 0.9134 - loss: 0.2145 - val_accuracy: 0.9092 - val_loss: 0.2317
Epoch 4/30
2500/2500 ━━━━━━━━━━━━━━━━━━━━ 148s 59ms/step - accuracy: 0.9240 - loss: 0.1906 - val_accuracy: 0.9262 - val_loss: 0.1853
Epoch 5/30
2500/2500 ━━━━━━━━━━━━━━━━━━━━ 158s 63ms/step - accuracy: 0.9310 - loss: 0.1719 - val_accuracy: 0.9287 - val_loss: 0.1795
Epoch 6/30
2500/2500 ━━━━━━━━━━━━━━━━━━━━ 149s 59ms/step - accuracy: 0.9390 - loss: 0.1546 - val_accuracy: 0.9367 - val_loss: 0.1648
Epoch 7/30
2500/2500 ━━━━━━━━━━━━━━━━━━━━ 272s 109ms/step - accuracy: 0.9423 - loss: 0.1454 - val_accuracy: 0.9380 - val_loss: 0.1649
Epoch 8/30
2500/2500 ━━━━━━━━━━━━━━━━━━━━ 139s 56ms/step - accuracy:

In [18]:
# Evaluate the model on the test dataset
test_loss, test_accuracy = model.evaluate(test_dataset)
print(f"Test accuracy: {test_accuracy * 100:.2f}%")

625/625 ━━━━━━━━━━━━━━━━━━━━ 28s 45ms/step - accuracy: 0.9209 - loss: 0.4425
Test accuracy: 92.17%
